<a href="https://colab.research.google.com/github/greyhound101/Orbuculum/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import keras
import keras.backend as K
from keras.applications import *
from keras.layers import *
from keras.models import Model
import tensorflow as tf
from keras.activations import *
from keras.models import Sequential
import numpy as np
import cv2
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
class SE(keras.layers.Layer):
    def __init__(self,factor,otr,**kwargs):
        super(SE, self).__init__(**kwargs)
        self.inr=int(otr/factor)
        self.otr=otr
        
    def get_config(self):
        base_config = super(SE, self).get_config()

    def build(self, input_shape):
        super(SE, self).build(input_shape)
        self.pool1=GlobalAveragePooling2D()
        self.compress=Dense(self.inr,activation='relu')
        self.expand=Dense(self.otr,activation='sigmoid')

    def call(self, img):
        y = self.pool1(img)
        y = self.compress(y)
        map = self.expand(y)
        map = tf.expand_dims(map,1)
        map = tf.expand_dims(map,2)
        return tf.math.multiply(img,map)
def load_model(last=True): 
  K.clear_session()   
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(5,5), input_shape=(30,30,3)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D(pool_size=(2, 2)))
  model.add(Conv2D(filters=64, kernel_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(SE(4,64))
    
  model.add(Conv2D(filters=32, kernel_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D(pool_size=(2, 2)))
  model.add(Conv2D(filters=64, kernel_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(SE(8,64))
    
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(rate=0.5))
  model.add(Dense(43, activation='softmax'))
  return model
model=load_model()
model.load_weights('/content/weights.hdf5')
a=cv2.imread('/content/image.ppm')
x1=int(input('Enter ROI x1'))
x2=int(input('Enter ROI x2'))
y1=int(input('Enter ROI y1'))
y2=int(input('Enter ROI y2'))
img=cv2.resize(a[x1:x2,y1:y2,:],(30,30), interpolation = cv2.INTER_AREA)
pre=model.predict(img.reshape(1,30,30,3)/255)
np.argmax(pre,1)